In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import seaborn
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from pycaret.classification import *
from pycaret.classification import evaluate_model
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import selenium
from selenium import webdriver
import random
from selenium.webdriver.common.by import By
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime

In [11]:
today = f"{datetime.now():%Y%m%d}"
df = pd.read_csv(f'data_{today}.csv', index_col = [0])

In [12]:
df

,Expected goals (xG),Total shots,Big chances,Big chances missed,Accurate passes,Accurate passes percentage,Fouls committed,Offsides,Corners,Shots off target,...,Successful dribbles percentage,score,posession,xG penalty,odds_checker_sum,0,1,2,odds_predict,final_score
0,0.89,7.0,1.0,0.0,77.0,0.06,-3.0,1.0,1.0,4.0,...,-0.31,0.0,20.0,NaN,1.091726,2.300000,2.387500,4.200000,0,0
1,-1.21,-5.0,-1.0,0.0,-54.0,-0.09,9.0,-1.0,-1.0,-1.0,...,-0.11,-1.0,-16.0,NaN,1.087449,4.750000,13.000000,1.250000,2,2
2,-0.74,-3.0,-1.0,0.0,-140.0,-0.11,-3.0,0.0,-1.0,0.0,...,-0.80,-2.0,-26.0,NaN,1.044048,20.000000,56.000000,1.024390,2,2
3,0.37,4.0,1.0,1.0,56.0,0.04,-1.0,-1.0,-1.0,3.0,...,0.04,-1.0,12.0,NaN,1.064560,3.500000,6.500000,1.600000,2,1
4,-2.94,-12.0,-3.0,0.0,-242.0,-0.33,-1.0,-1.0,-4.0,0.0,...,0.60,-4.0,-50.0,NaN,0.970938,226.000000,201.000000,1.040000,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,0.10,0.0,0.0,0.0,31.0,0.00,1.0,0.0,-1.0,1.0,...,0.67,0.0,8.0,NaN,1.082870,2.500000,3.200000,2.700000,0,0
241,-0.24,-4.0,0.0,0.0,39.0,0.02,-2.0,0.0,-3.0,1.0,...,0.23,0.0,8.0,NaN,1.088785,2.150000,4.400000,2.522727,0,0
242,-0.68,-9.0,1.0,0.0,-76.0,-0.15,-2.0,0.0,-4.0,-3.0,...,0.18,0.0,-20.0,NaN,1.094355,2.647059,5.500000,1.870000,2,0
243,0.66,3.0,1.0,0.0,206.0,0.18,5.0,-1.0,1.0,2.0,...,0.61,0.0,42.0,0.78,1.080018,3.100000,1.533333,9.500000,1,0


In [ ]:
grid = setup(data=df, target='final_score', session_id = 123, remove_multicollinearity = True,
             feature_selection = True, feature_selection_threshold = 0.6)

In [14]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.6725,0.8547,0.6558,0.6842,0.6638,0.5032,0.5145,0.1160
lightgbm,Light Gradient Boosting Machine,0.6722,0.8517,0.6531,0.6720,0.6619,0.5008,0.5094,0.0360
gbc,Gradient Boosting Classifier,0.6614,0.8347,0.6445,0.6778,0.6543,0.4859,0.4966,0.2060
et,Extra Trees Classifier,0.6078,0.8167,0.5813,0.6044,0.5887,0.4015,0.4118,0.1880
svm,SVM - Linear Kernel,0.6023,0.0000,0.5785,0.6100,0.5813,0.3897,0.4100,0.0320
rf,Random Forest Classifier,0.6023,0.8180,0.5724,0.5956,0.5853,0.3928,0.4026,0.1970
ada,Ada Boost Classifier,0.5843,0.7915,0.5722,0.6059,0.5726,0.3740,0.3862,0.0450
nb,Naive Bayes,0.5784,0.8312,0.6064,0.7327,0.5643,0.3916,0.4681,0.0120
dt,Decision Tree Classifier,0.5732,0.6856,0.5665,0.6155,0.5690,0.3601,0.3751,0.0080
lda,Linear Discriminant Analysis,0.5667,0.7238,0.5687,0.6216,0.5691,0.3547,0.3724,0.0120


In [347]:
xgb = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7143,0.8940,0.6556,0.6990,0.7011,0.5447,0.5495
1,0.7143,0.8399,0.6444,0.6964,0.6912,0.5410,0.5556
2,0.7857,0.9408,0.7111,0.7602,0.7670,0.6585,0.6643
3,0.5714,0.7667,0.5667,0.5893,0.5758,0.3538,0.3566
4,0.5000,0.7762,0.4833,0.4779,0.4785,0.2403,0.2462
5,0.7143,0.9232,0.7222,0.7619,0.7143,0.5758,0.5938
6,0.7143,0.9089,0.6944,0.7821,0.6744,0.5484,0.5838
7,0.8462,0.9626,0.8333,0.8615,0.8469,0.7636,0.7707
8,0.6923,0.7956,0.6667,0.7051,0.6923,0.5229,0.5278


In [15]:
tuned = tune_model(best)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7778,0.9434,0.7444,0.7654,0.7569,0.6571,0.6704
1,0.7647,0.8893,0.7302,0.7794,0.7679,0.6364,0.6398
2,0.7647,0.9202,0.7738,0.8235,0.7583,0.6440,0.6762
3,0.7059,0.8910,0.7103,0.7515,0.7137,0.5503,0.5592
4,0.5294,0.8084,0.5222,0.5336,0.5289,0.2880,0.2895
5,0.7647,0.9086,0.7556,0.7647,0.7647,0.6458,0.6458
6,0.8235,0.9750,0.8111,0.8382,0.8195,0.7330,0.7448
7,0.6471,0.8155,0.6444,0.7017,0.6600,0.4715,0.4790
8,0.5882,0.6907,0.5778,0.5794,0.5753,0.3770,0.3831


In [16]:
evaluate_model(tuned)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

<Figure size 432x288 with 0 Axes>

In [19]:
predicted_df = predict_model(tuned, raw_score = True)
predicted_df = predicted_df[['0','1','2'] + list(predicted_df.columns[-5:])]
bo_po_dha = list(zip(predicted_df.columns[0:3],predicted_df.columns[-3:],['draw','home','away']))
for book_odds, pred_odds, dha in bo_po_dha:
    predicted_df[dha] = predicted_df.apply(lambda x: kelly(x[pred_odds], x[book_odds], 1), axis=1)
predicted_df.to_csv('pred_profits.csv')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.5676,0.7254,0.5470,0.6212,0.5820,0.3341,0.3435


In [20]:
predicted_df

,0,1,2,final_score,Label,Score_0,Score_1,Score_2,draw,home,away
0,2.300,3.100000,3.200,0,0,0.9714,0.0096,0.0190,0.9494,no value,no value
1,4.600,1.292857,13.000,1,1,0.0435,0.9560,0.0005,no value,0.8058,no value
2,3.800,1.400000,9.500,1,0,0.6683,0.3213,0.0104,0.5498,no value,no value
3,2.625,2.050000,5.125,1,1,0.2491,0.6171,0.1338,no value,0.2524,no value
4,2.500,4.550000,2.150,2,0,0.9835,0.0046,0.0119,0.9725,no value,no value
...,...,...,...,...,...,...,...,...,...,...,...
69,3.275,1.775000,4.500,2,1,0.3589,0.5916,0.0495,0.0771,0.0646,no value
70,5.900,1.166667,19.000,1,1,0.0469,0.9525,0.0006,no value,0.6675,no value
71,2.375,2.550000,3.800,0,0,0.8872,0.1040,0.0089,0.8052,no value,no value
72,3.100,3.400000,2.165,2,0,0.8109,0.1550,0.0341,0.7209,no value,no value


In [18]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'